# Capture video and stream it to Kafka

### Using OpenCV and kafka-python
![opencv](http://www.decom.ufop.br/imobilis/wp-content/uploads/2015/03/OpenCV_Logo.png) ![kafka](http://www.godatafy.com/wp-content/uploads/2015/07/tech-apachekafka-logo.jpg) ![python](https://jsantanna.files.wordpress.com/2016/07/python-logo.png)

In [11]:
# Import video, kafka and other libs
from kafka import KafkaProducer
import numpy as np
import cv2
import datetime, pickle, base64, json

In [12]:
# Setup topic camera device 
kafkabrokers='wn0-kafka2.e4ix3a2tssaedkygl5ugloqjrb.cx.internal.cloudapp.net:9092'
topic='camera-stream'
cameraId=0

# Create Kafka Producer
producer = KafkaProducer(bootstrap_servers=kafkabrokers, api_version=(0,10),
                         key_serializer=lambda k: k.encode('utf-8'),
                         value_serializer=lambda v: json.dumps(v).encode('utf-8'))
num_partitions = len(producer.partitions_for(topic))

In [13]:
# Open Camera stream
cap = cv2.VideoCapture(int(cameraId))
lsec = datetime.datetime.utcnow().second ; fc = 0

# Start capturing images and sending to Kafka
while(cap.isOpened()):
    # read frame and resize properly
    ret, frame = cap.read()
    frame = cv2.resize(frame, (0,0), fx=0.4, fy=0.4)
    
    # prepare message payload
    msg_payload = {
        'cameraId': cameraId,
        'frameTS': str(datetime.datetime.utcnow()),
        'frameB64': base64.standard_b64encode(pickle.dumps(frame)).decode("utf-8")
        }
    
    # send message to kafka and wait for frame to be delivered in Kafka
    producer.send(topic, partition = cameraId%num_partitions, 
                  key = str(cameraId)+'|'+msg_payload['frameTS'], value = msg_payload)
    producer.flush()
    
    # show image being captured and wait for key to stop capturing
    cv2.imshow('camera', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        # close capturing and windows
        cap.release()
        cv2.destroyAllWindows()
